## CNN DAILY/MAIL DATASET

In [6]:
import config
from datasets import load_dataset
import os

os.makedirs("data")

In [9]:
def preprocess_function_cnn_dailymail(
        examples,
        tokenizer,
        max_input_length: int = config.MAX_INPUT_LENGTH,
        max_target_length: int = config.MAX_TARGET_LENGTH,
    ):
        prefix = "summarize: "
        inputs = [prefix + doc for doc in examples["article"]]
        model_inputs = tokenizer(
            inputs, max_length=max_input_length, truncation=True, padding=True
        )

        # Setup the tokenizer for targets
        labels = tokenizer(
            text_target=examples["highlights"],
            max_length=max_target_length,
            truncation=True,
        )

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

# data_dir = "data"

cnn_data_train = load_dataset(
    "cnn_dailymail",'3.0.0', split=f"train[:{config.PERCENT_DATA}%]"
)
cnn_data_test = load_dataset(
    "cnn_dailymail",'3.0.0' , split=f"test[:{config.PERCENT_DATA}%]"
)
cnn_data_val = load_dataset(
    "cnn_dailymail",'3.0.0',split=f"validation[:{config.PERCENT_DATA}%]"
)

Generating test split: 100%|██████████| 11490/11490 [00:00<00:00, 81273.17 examples/s]


In [11]:
import torch
from transformers import AutoTokenizer
from transformers import T5ForConditionalGeneration

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name, legacy=False)
def preprocess_function_summary(examples,max_input_length:int=config.MAX_INPUT_LENGTH,max_target_length:int=config.MAX_TARGET_LENGTH):
        prefix = "summarize: "
        inputs = [prefix + doc for doc in examples["article"]]
        model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding=True)

        # Setup the tokenizer for targets
        labels = tokenizer(text_target=examples["highlights"], max_length=max_target_length, truncation=True)

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

tokenized_datasets_train = cnn_data_train.map(preprocess_function_summary, batched=True,remove_columns=['article','highlights','id'],batch_size=config.TOKENIZE_BATCH_SIZE)
tokenized_datasets_val = cnn_data_val.map(preprocess_function_summary, batched=True,remove_columns=['article','highlights','id'],batch_size=config.TOKENIZE_BATCH_SIZE)
tokenized_datasets_test = cnn_data_test.map(preprocess_function_summary, batched=True,remove_columns=['article','highlights','id'],batch_size=config.TOKENIZE_BATCH_SIZE)

Map: 100%|██████████| 11490/11490 [00:05<00:00, 2004.15 examples/s]


## ARXIV DATASET (TOO BIG PLANNING TO DISCARD)

In [20]:
# from datasets import load_dataset
arxiv_link = "https://github.com/armancohan/long-summarization/tree/master?tab=readme-ov-file"
# dataset = load_dataset("arxiv_dataset",trust_remote_code=True)

## PubMed DATASET

In [23]:
# import gdown
# pubmed_url = "https://archive.org/download/armancohan-long-summarization-paper-code/pubmed-dataset.zip"
# https://huggingface.co/datasets/scientific_papers?row=0
# output = 'pubmed.zip'
# gdown.download(pubmed_url, output, quiet=False)

In [3]:
import os
pubmed_dir = "pubmed-dataset"

for file in os.listdir(pubmed_dir):
    # print(file)
    if file.endswith(".txt"):
        txt_file = os.path.join(pubmed_dir, file)

test.txt
vocab
val.txt
train.txt


In [4]:
from datasets import load_dataset

pubmed_dataset = load_dataset(
    "scientific_papers","pubmed"
)

/home/athekunal/GQA/Enhancement-in-GQA/gqa-env/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating test split: 100%|██████████| 6658/6658 [00:01<00:00, 5007.96 examples/s]


In [5]:
pubmed_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6658
    })
})

## MEDIA SUM DATASET (TOO LARGE PLANNING TO DISCARD)

Link to [data](https://aclanthology.org/2021.naacl-main.474/)

## MULTI-NEWS DATASET

In [1]:
from datasets import load_dataset

multi_news_dataset = load_dataset("multi_news")

/home/athekunal/GQA/Enhancement-in-GQA/gqa-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
multi_news_dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

## WMT DATASET

In [4]:
# train_en_link = "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.en"
# train_de_link = "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.de"

# test_en_link = "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2014.en"
# test_de_link = "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2014.de"

wmt_dataset = load_dataset("stas/wmt14-en-de-pre-processed")

In [7]:
wmt_dataset['test']['translation'][0]['en']

'Obama receives Netanyahu'

## TRIVIA-QA DATASET

## FINAL FUNCTIONS

In [ ]:

dataset_type = ["cnn_dailymail","pubmed","multi_news","wmt14","triviaqa"]

def preprocess_function(examples,dataset_name:str,max_input_length:int=config.MAX_INPUT_LENGTH,max_target_length:int=config.MAX_TARGET_LENGTH):
        if dataset_name == "cnn_dailymail":
            prefix = "summarize: "
            inputs = [prefix + doc for doc in examples["article"]]
            model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding=True)

            # Setup the tokenizer for targets
            labels = tokenizer(text_target=examples["highlights"], max_length=max_target_length, truncation=True)

            model_inputs["labels"] = labels["input_ids"]
            return model_inputs
        elif dataset_name == "pubmed":
            prefix = "summarize: "
            NotImplementedError()
        elif dataset_name == "multi_news":
            prefix = "summarize: "
            inputs = [prefix + doc for doc in examples["document"]]
            model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding=True)

            # Setup the tokenizer for targets
            labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

            model_inputs["labels"] = labels["input_ids"]
            return model_inputs
        elif dataset_name == "wmt14":
            prefix = "translate german to english: "
            inputs = [prefix + doc for doc in examples["translation"]['en']]
            model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding=True)

            # Setup the tokenizer for targets
            text_targets = [ex['de'] for ex in examples["translation"]]
            labels = tokenizer(text_target=text_targets, max_length=max_target_length, truncation=True)

            model_inputs["labels"] = labels["input_ids"]
            return model_inputs
tokenized_datasets_train = cnn_data_train.map(preprocess_function, batched=True,remove_columns=['article','highlights','id'],batch_size=config.TOKENIZE_BATCH_SIZE)
tokenized_datasets_val = cnn_data_val.map(preprocess_function, batched=True,remove_columns=['article','highlights','id'],batch_size=config.TOKENIZE_BATCH_SIZE)
tokenized_datasets_test = cnn_data_test.map(preprocess_function, batched=True,remove_columns=['article','highlights','id'],batch_size=config.TOKENIZE_BATCH_SIZE)